# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'mgmt-467-471819' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `mgmt-467-471819.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country AS country,
  City AS city,
  State AS state,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `mgmt-467-471819.lab1_foundation.superstore`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country | city | state | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/mgmt-467-471819/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `mgmt-467-471819.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `mgmt-467-47888.lab1_foundation.superstore`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?
Yes, this matched my expectations.


In [ ]:
query_string = """
SELECT
    DISTINCT sub_category
FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
WHERE
    region = 'West'
ORDER BY
    sub_category ASC
LIMIT 100
"""
results_df = query_job.to_dataframe()
display(results_df)

,order_id,customer_name,product_name,sales,profit
0,CA-2015-154900,Sung Shariari,Avery 518,3.15,1.5120
1,CA-2015-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2016-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2016-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,CA-2016-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
5,CA-2016-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
6,CA-2016-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,CA-2016-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
8,CA-2015-158792,Brian Dahlen,Staples,22.20,10.4340
9,CA-2016-141082,Fred McMath,Avery 517,3.69,1.7343


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `mgmt-467-47888.lab_foundation.superstore`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [ ]:
query_string = """
SELECT
    customer_id,
    SUM(profit) AS total_profit
FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
GROUP BY
    customer_id
ORDER BY
    total_profit DESC
LIMIT 10
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,customer_id,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

In [ ]:
query_string = """
SELECT
    ship_mode,
    COUNT(*) AS order_count
FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
WHERE
    category = 'Technology'
GROUP BY
    ship_mode
ORDER BY
    order_count DESC
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,ship_mode,order_count
0,Standard Class,1082
1,Second Class,366
2,First Class,301
3,Same Day,98


Sanity Checks:
"SELECT
  COUNT(*) AS total_technology_orders
FROM
  `mgmt-467-471819.lab1_foundation.superstore_clean`
WHERE
  category = 'Technology';"
  "SELECT
  COUNT(*) AS standard_class_tech_orders
FROM
  `mgmt-467-471819.lab1_foundation.superstore_clean`
WHERE
  category = 'Technology' AND ship_mode = 'Standard Class';"

## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [ ]:
query_string = """
SELECT
    FORMAT_DATE('%Y-%m', DATE_TRUNC(order_date, MONTH)) AS year_month,
    SUM(sales) AS monthly_revenue
FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
WHERE
    order_date >= DATE_TRUNC(DATE_SUB((SELECT MAX(order_date) FROM `mgmt-467-471819.lab1_foundation.superstore_clean`), INTERVAL 11 MONTH), MONTH)
GROUP BY
    year_month
ORDER BY
    year_month ASC
LIMIT 100;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,year_month,monthly_revenue
0,2017-01,43971.3740
1,2017-02,20301.1334
2,2017-03,58872.3528
3,2017-04,36521.5361
4,2017-05,44261.1102
5,2017-06,52981.7257
6,2017-07,45264.4160
7,2017-08,63120.8880
8,2017-09,87866.6520
9,2017-10,77776.9232


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [ ]:
query_string = """
SELECT
    sub_category,
    SUM(profit) AS total_profit
FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
GROUP BY
    sub_category
HAVING
    total_profit < 0
ORDER BY
    total_profit ASC;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,sub_category,total_profit
0,Tables,-17725.4811
1,Bookcases,-3472.5560
2,Supplies,-1189.0995


The HAVING clause is used because it filters groups after an aggregation function like SUM() has been applied, whereas a WHERE clause filters individual rows before aggregation.

## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [ ]:
query_string = """
WITH products_dimension AS (
  SELECT DISTINCT
    product_id,
    product_name
  FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
)
SELECT
  p.product_id,
  p.product_name,
  SUM(s.sales) AS total_sales
FROM
  `mgmt-467-471819.lab1_foundation.superstore_clean` AS s
JOIN
  products_dimension AS p ON s.product_id = p.product_id
GROUP BY
  p.product_id, p.product_name
ORDER BY
  total_sales DESC
LIMIT 20;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61599.8240
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.3840
2,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.4800
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.5760
4,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.4790
5,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,19024.5000
6,TEC-CO-10001449,Hewlett Packard LaserJet 3310 Copier,18839.6860
7,TEC-MA-10001127,HP Designjet T520 Inkjet Large Format Printer ...,18374.8950
8,OFF-BI-10004995,GBC DocuBind P400 Electric Binding System,17965.0680
9,OFF-SU-10000151,High Speed Automatic Electric Letter Opener,17030.3120


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

This query uses two Common Table Expressions (CTEs) to first calculate total sales for each state, then rank those states within their respective regions, and finally filters to show only the top 3 for each region. The first CTE, state_sales, aggregates sales by state and region. The second CTE, ranked_state_sales, applies the RANK() window function to assign a sales rank to each state, partitioning the calculation by region so the ranking restarts for each one. The final SELECT statement then pulls from the second CTE, keeping only the rows where the rank is 3 or less.


In [ ]:
query_string = """
WITH state_sales AS (
  -- First, calculate the total sales for each State and Region combination.
  SELECT
    Region,
    State,
    SUM(Sales) AS total_sales
  FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
  GROUP BY
    Region, State
),
ranked_state_sales AS (
  -- Second, rank the states within each region based on their total sales.
  SELECT
    Region,
    State,
    total_sales,
    RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC) AS sales_rank
  FROM
    state_sales
)
-- Finally, select only the rows where the rank is in the top 3 for each region.
SELECT
  Region,
  State,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  Region, sales_rank;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Region,State,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

In [ ]:
query_string = """
WITH yr_sales AS (
  -- First, calculate total sales for each sub-category, for each year.
  SELECT
    sub_category,
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS total_sales
  FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM order_date) IN (2023, 2024)
  GROUP BY
    sub_category, year
)
-- Self-join or pivot to get 2023 and 2024 sales on the same row
SELECT
  s2024.sub_category,
  COALESCE(s2023.total_sales, 0) AS sales_2023,
  COALESCE(s2024.total_sales, 0) AS sales_2024,
  (COALESCE(s2024.total_sales, 0) - COALESCE(s2023.total_sales, 0)) AS yoy_delta
FROM
  (SELECT * FROM yr_sales WHERE year = 2024) AS s2024
FULL OUTER JOIN
  (SELECT * FROM yr_sales WHERE year = 2023) AS s2023
  ON s2024.sub_category = s2023.sub_category
ORDER BY
  yoy_delta DESC
LIMIT 5;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,sub_category,sales_2023,sales_2024,yoy_delta


Missing Years for a Sub-Category: A sub-category might have sales in 2024 but not in 2023 (making it a new growth area), or vice-versa. A simple INNER JOIN would exclude these, but a FULL OUTER JOIN combined with COALESCE(column, 0) ensures they are included in the analysis with a zero value for the missing year.

No Data for Required Years: The entire dataset might lack data for 2023 or 2024. The query would run but return an empty result, which could be misleading; to handle this, you could first run a sanity check like SELECT DISTINCT EXTRACT(YEAR FROM order_date) FROM your_table to confirm the years exist before running the main analysis.

## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

In [19]:
query_string = """
WITH subcat_sales AS (
  SELECT
    Region,
    sub_category,
    SUM(Sales) AS total_sales
  FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
  GROUP BY
    Region, sub_category
),
ranked_sales AS (
    SELECT
        Region,
        sub_category,
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC) as rn
    FROM subcat_sales
)
SELECT
  Region,
  sub_category,
  total_sales
FROM
  ranked_sales
WHERE
  rn = 1
ORDER BY
  Region;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,Region,sub_category,total_sales
0,Central,Chairs,85230.646
1,East,Phones,100614.982
2,South,Phones,58304.438
3,West,Chairs,101781.328


ROW_NUMBER() is used to ensure only a single row is returned per region, even if multiple sub-categories have the exact same total sales. In contrast, RANK() would assign the same rank to any tied sub-categories, potentially causing the query to return more than one "top" result per region, which violates the request for a single sub-category.

### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [21]:
query_string = """
WITH phones_yearly_sales AS (
  -- First, aggregate sales for the 'Phones' sub-category by year
  SELECT
    EXTRACT(YEAR FROM order_date) AS year,
    SUM(sales) AS yearly_revenue
  FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
  WHERE
    sub_category = 'Phones'
  GROUP BY
    year
),
sales_with_lag AS (
  -- Use the LAG window function to get the previous year's revenue
  SELECT
    year,
    yearly_revenue,
    LAG(yearly_revenue, 1) OVER (ORDER BY year) AS prev_revenue
  FROM
    phones_yearly_sales
)
-- Calculate the final YoY growth percentage, safely handling the first year
SELECT
  year,
  yearly_revenue,
  prev_revenue,
  SAFE_DIVIDE(100.0 * (yearly_revenue - prev_revenue), prev_revenue) AS yoy_pct
FROM
  sales_with_lag
ORDER BY
  year ASC;
  """
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,year,yearly_revenue,prev_revenue,yoy_pct
0,2014,77390.806,NaN,NaN
1,2015,68313.702,77390.806,-11.728918
2,2016,78962.030,68313.702,15.587397
3,2017,105340.516,78962.030,33.406545


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [22]:
query_string = """
WITH monthly_sales AS (
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sales) AS monthly_revenue
  FROM
    `mgmt-467-471819.lab1_foundation.superstore_clean`
  WHERE
    segment = 'Corporate'
  GROUP BY
    month
)
SELECT
  month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
FROM
  monthly_sales
ORDER BY
  month ASC;
"""
query_job = client.query(query_string)
results_df = query_job.to_dataframe()
display(results_df)

,month,monthly_revenue,ma_3
0,2014-01-01,1701.5280,1701.528000
1,2014-02-01,1183.6680,1442.598000
2,2014-03-01,11106.7990,4663.998333
3,2014-04-01,14131.7290,8807.398667
4,2014-05-01,9142.0000,11460.176000
5,2014-06-01,3970.9140,9081.547667
6,2014-07-01,10032.9880,7715.300667
7,2014-08-01,7451.7740,7151.892000
8,2014-09-01,15507.7450,10997.502333
9,2014-10-01,12637.6780,11865.732333


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
[PASTE ERROR MESSAGE and the exact SQL here]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

SELECT
  order_id,
  SUM(sales) AS total_sales
FROM
  `mgmt-467-471819.lab1_foundation.superstore_clean`
GROUP BY
  order_id
HAVING
  total_sales > 5000;
The corrected SQL replaces WHERE with HAVING because HAVING is required to filter on the results of an aggregate function like SUM(). Using WHERE total_sales > 5000 is invalid as the total_sales alias is not available until after the GROUP BY operation has completed.

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

Add a Partition Filter: The most impactful change is to filter directly on the partition column instead of using a function. Using EXTRACT(YEAR FROM order_date) forces BigQuery to scan the entire table. By changing the WHERE clause to WHERE order_date BETWEEN '2024-01-01' AND '2024-12-31', you enable partition pruning, allowing BigQuery to scan only the data for 2024, dramatically reducing bytes processed.

Prune Columns: The query uses SELECT *, which is highly inefficient as it reads every column from the table. You should explicitly list only the columns you need in your final output (e.g., SELECT order_id, order_date, sales, profit). This practice, known as column pruning, directly reduces the amount of data scanned and processed by the query.

Create Pre-Aggregated Summary Tables: If you frequently need to analyze sales by category and date, it's more efficient to pre-aggregate the data. You could create a daily or monthly summary table that contains total sales, profit, and order counts for each category. Your analytical queries would then run against this much smaller summary table instead of the huge raw superstore_clean table, making them significantly faster and cheaper.

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

Query 1: Slicing by Region and Time

SELECT
  region,
  FORMAT_DATE('%Y-%m', order_date) AS year_month,
  SUM(sales) AS total_sales,
  AVG(discount) AS average_discount,
  SUM(profit) AS total_profit
FROM
  `mgmt-467-471819.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  region, year_month
ORDER BY
  region, year_month;
Query 2: Controlling for Ship Mode

SELECT
  ship_mode,
  COUNT(*) AS number_of_orders,
  SUM(sales) AS total_sales,
  AVG(discount) AS average_discount,
  SUM(profit) AS total_profit
FROM
  `mgmt-467-471819.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  ship_mode
ORDER BY
  total_profit ASC;
To compare these outcomes, first, analyze the region and time query to see if the negative profit for 'Tables' is consistent or isolated to specific areas or periods; this could reveal that the problem is driven by local promotions rather than a universal issue. Next, run the ship mode query to determine if a particular shipping method is disproportionately responsible for the losses, which would suggest that logistics and fulfillment costs, not just discounts, are a critical factor. Together, these analyses will help you move from observing the problem to understanding its specific drivers, allowing for more targeted business recommendations.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

Based on the analysis of the 'superstore_clean' data, which revealed significant year-over-year revenue growth in categories like 'Copiers' while identifying major profit loss in the 'Tables' sub-category, generate a 4-sentence executive summary. The summary will state what changed, identify the likely drivers (high-margin tech vs. low-margin furniture), recommend an action for Sales and Operations leaders, and define a key metric to monitor for the unprofitable category.


### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---